<a href="https://colab.research.google.com/github/UsirkepallyVarsha/StudyMate/blob/main/Streamlit_App_with_Gemini_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install gTTS

In [25]:
!pip install streamlit

In [26]:
!pip install python-dotenv

In [33]:
import streamlit as st
import google.generativeai as genai
import os
from dotenv import load_dotenv
import time
from gtts import gTTS
import io

# --- Colab Specific Imports and Check ---
# This block attempts to import Colab-specific modules.
# If successful, it means the code is likely running in Colab.
try:
    from google.colab import userdata
    from pyngrok import ngrok, conf
    IS_COLAB = True
except ImportError:
    IS_COLAB = False
    # If not in Colab, ensure pyngrok is imported for local execution
    try:
        from pyngrok import ngrok, conf
    except ImportError:
        # Stop execution if pyngrok is not installed and we are not in Colab
        st.error("Please install pyngrok: pip install pyngrok")
        st.stop()


# --- Configuration & Setup ---

# Load environment variables (for local runs)
if not IS_COLAB:
    load_dotenv()

# Gemini API Configuration
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if IS_COLAB:
    try:
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    except Exception as e:
        st.error(f"Error accessing Colab secret GOOGLE_API_KEY: {e}")

if not GOOGLE_API_KEY:
    st.error("Google API Key not found. Please set GOOGLE_API_KEY in Colab secrets or your .env file.")
    st.stop()
genai.configure(api_key=GOOGLE_API_KEY)

# Ngrok Configuration
NGROK_AUTH_TOKEN = None
if IS_COLAB:
    st.info("Running in Google Colab. Setting up ngrok for public access...")
    try:
        NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    except Exception as e:
        st.error(f"Error accessing Colab secret NGROK_AUTH_TOKEN: {e}")
else:
    NGROK_AUTH_TOKEN = os.getenv("NGROK_AUTH_TOKEN")

if NGROK_AUTH_TOKEN:
    try:
        # ngrok will not work on Colab without an auth token
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        ngrok.kill()  # Kill any existing ngrok tunnels

        # Streamlit defaults to port 8501
        public_url = ngrok.connect(addr="8501", proto="http")
        st.sidebar.success(f"**Public URL:** [{public_url}](<{public_url}>)")
        st.sidebar.info("Share this URL to allow others to access your app.")

        if IS_COLAB:
            print(f"Your Streamlit app is publicly available at: {public_url}")

    except Exception as e:
        st.sidebar.error(f"Ngrok connection failed: {e}. Check your NGROK_AUTH_TOKEN and ensure ngrok is properly configured.")
        st.stop()
else:
    st.sidebar.warning("Ngrok Auth Token not found. Your app will only be accessible locally.")


# --- Streamlit App UI ---

st.set_page_config(page_title="Gemini Multipurpose AI", page_icon="🤖", layout="wide")

st.title("Gemini Multipurpose AI 🤖")
st.markdown("A versatile AI assistant for chat, summarization, quizzes, and more!")

# Sidebar for controls and info
st.sidebar.header("AI Settings")
model_name = st.sidebar.selectbox(
    "Choose Gemini Model:",
    ("gemini-pro", "gemini-pro-vision"),
    help="Gemini-pro for text generation. Gemini-pro-vision for multimodal (text+image) if implemented."
)

temperature = st.sidebar.slider(
    "Temperature (Creativity):",
    min_value=0.0,
    max_value=1.0,
    value=0.7,
    step=0.05,
    help="Higher values make the output more random and creative. Lower values make it more focused and deterministic."
)

max_output_tokens = st.sidebar.slider(
    "Max Output Tokens:",
    min_value=50,
    max_value=2048,
    value=500,
    step=50,
    help="The maximum number of tokens to generate in the response."
)

st.sidebar.markdown("---")
st.sidebar.header("App Features")
selected_feature = st.sidebar.radio(
    "Select Feature:",
    ["Interactive Chatbot", "Quick Summarizer", "Quiz Mode", "Notes Maker (WIP)", "Visualizer (WIP)", "PDF Tools (WIP)", "About"]
)

st.sidebar.markdown("---")
st.sidebar.header("History")
if st.sidebar.button("Clear All History", type="secondary"):
    st.session_state.messages = []
    st.session_state.quiz_started = False
    st.session_state.quiz_questions = []
    st.session_state.quiz_score = 0
    st.session_state.current_question_idx = 0
    st.rerun()

st.sidebar.markdown("[Source Code (Placeholder)](https://github.com/your-repo/your-app)")

# Initialize the Gemini model with selected parameters
try:
    model = genai.GenerativeModel(
        model_name=model_name,
        generation_config=genai.types.GenerationConfig(
            temperature=temperature,
            max_output_tokens=max_output_tokens,
        )
    )
except Exception as e:
    st.error(f"Failed to initialize Gemini model: {e}")
    st.stop()

# --- Feature Implementations ---

if selected_feature == "Interactive Chatbot":
    st.header("💬 Interactive Chatbot")
    st.markdown("Engage in a free-form conversation with Gemini.")

    if "messages" not in st.session_state:
        st.session_state.messages = []

    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
            if message["role"] == "assistant" and "audio_data" in message:
                st.audio(message["audio_data"], format='audio/mpeg')

    if prompt := st.chat_input("What's on your mind?"):
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        with st.spinner("Thinking..."):
            try:
                history_for_gemini = []
                for msg in st.session_state.messages[:-1]:
                    role = "user" if msg["role"] == "user" else "model"
                    history_for_gemini.append({"role": role, "parts": [msg["content"]]})

                response = model.generate_content(
                    contents=history_for_gemini + [{"role": "user", "parts": [prompt]}]
                )

                gemini_response = response.text

                message_placeholder = st.empty()
                full_response = ""
                for chunk in gemini_response.split(" "):
                    full_response += chunk + " "
                    time.sleep(0.02)
                    message_placeholder.markdown(full_response + "▌")
                message_placeholder.markdown(full_response)

                try:
                    tts = gTTS(text=gemini_response, lang='en')
                    audio_fp = io.BytesIO()
                    tts.save(audio_fp)
                    audio_fp.seek(0)
                    st.session_state.messages.append({"role": "assistant", "content": gemini_response, "audio_data": audio_fp.read()})
                    st.audio(audio_fp)
                except Exception as audio_err:
                    st.warning(f"Could not generate audio for response: {audio_err}")
                    st.session_state.messages.append({"role": "assistant", "content": gemini_response})

            except genai.types.BlockedPromptException as e:
                st.warning(f"Response blocked due to safety concerns. Please try a different query.")
                st.error(f"Details: {e.safety_ratings}")
                st.session_state.messages.append({"role": "assistant", "content": "*(Response blocked due to safety concerns.)*"})
            except Exception as e:
                st.error(f"An error occurred: {e}")
                st.warning("Could not get a response from Gemini. Please try again.")
                st.session_state.messages.append({"role": "assistant", "content": "*(Error generating response.)*"})

elif selected_feature == "Quick Summarizer":
    st.header("📝 Quick Summarizer")
    st.markdown("Enter text to get a concise summary.")
    summary_text = st.text_area("Enter text to summarize:", height=250, placeholder="Paste your article, document, or paragraph here...")
    if st.button("Generate Summary"):
        if summary_text:
            with st.spinner("Summarizing..."):
                try:
                    prompt = f"Summarize the following text concisely and accurately:\n\n{summary_text}"
                    response = model.generate_content(prompt)
                    summary = response.text
                    st.subheader("Summary:")
                    st.write(summary)
                except Exception as e:
                    st.error(f"Error summarizing text: {e}")
        else:
            st.warning("Please enter some text to summarize.")

elif selected_feature == "Quiz Mode":
    st.header("❓ Quiz Mode")
    st.markdown("Generate a quiz from provided text. (Basic multiple choice)")
    if "quiz_started" not in st.session_state:
        st.session_state.quiz_started = False
        st.session_state.quiz_questions = []
        st.session_state.quiz_score = 0
        st.session_state.current_question_idx = 0

    if not st.session_state.quiz_started:
        quiz_text = st.text_area("Enter text to generate a quiz from:", height=200, placeholder="Provide text for the quiz.")
        num_questions = st.slider("Number of questions:", min_value=1, max_value=5, value=3)
        if st.button("Generate Quiz"):
            if quiz_text:
                with st.spinner("Generating quiz questions..."):
                    try:
                        quiz_prompt = f"""Generate a {num_questions}-question multiple-choice quiz based on the following text.
                        For each question, provide 4 options (A, B, C, D) and clearly indicate the correct answer.
                        Format:
                        Q1: [Question Text]
                        A) [Option A]
                        B) [Option B]
                        C) [Option C]
                        D) [Option D]
                        Correct Answer: [A/B/C/D]
                        Text:
                        {quiz_text}"""
                        quiz_response = model.generate_content(quiz_prompt)
                        quiz_raw_text = quiz_response.text
                        questions = []
                        current_question = {}
                        lines = quiz_raw_text.split('\n')
                        for line in lines:
                            line = line.strip()
                            if line.startswith("Q") and ":" in line:
                                if current_question: questions.append(current_question)
                                current_question = {"question": line, "options": [], "correct": None}
                            elif line.startswith(("A)", "B)", "C)", "D)")) and current_question:
                                current_question["options"].append(line)
                            elif line.startswith("Correct Answer:") and current_question:
                                current_question["correct"] = line.split(":")[1].strip()
                        if current_question: questions.append(current_question)
                        if questions:
                            st.session_state.quiz_questions = questions
                            st.session_state.quiz_started = True
                            st.session_state.quiz_score = 0
                            st.session_state.current_question_idx = 0
                            st.rerun()
                        else:
                            st.error("Could not parse quiz questions. Please try again.")
                    except Exception as e:
                        st.error(f"Error generating quiz: {e}")
            else:
                st.warning("Please enter text to generate a quiz.")
    else:
        if st.session_state.current_question_idx < len(st.session_state.quiz_questions):
            q = st.session_state.quiz_questions[st.session_state.current_question_idx]
            st.subheader(f"Question {st.session_state.current_question_idx + 1}/{len(st.session_state.quiz_questions)}")
            st.markdown(q["question"])
            display_options = [opt.split(') ')[1] if ') ' in opt else opt for opt in q["options"]]
            user_answer = st.radio("Choose your answer:", display_options, key=f"q{st.session_state.current_question_idx}")
            if st.button("Submit Answer"):
                correct_option_text = [opt.split(') ')[1] for opt in q["options"] if opt.startswith(q["correct"])][0] if q["correct"] else "N/A"
                if user_answer == correct_option_text:
                    st.success("Correct! 🎉")
                    st.session_state.quiz_score += 1
                else:
                    st.error(f"Incorrect. The correct answer was: {correct_option_text} 😔")
                time.sleep(1.5)
                st.session_state.current_question_idx += 1
                st.rerun()
        else:
            st.subheader("Quiz Completed! 🥳")
            st.success(f"You scored {st.session_state.quiz_score} out of {len(st.session_state.quiz_questions)}!")
            if st.button("Restart Quiz"):
                st.session_state.quiz_started = False
                st.session_state.quiz_questions = []
                st.session_state.quiz_score = 0
                st.session_state.current_question_idx = 0
                st.rerun()

elif selected_feature == "Notes Maker (WIP)":
    st.header("📝 Notes Maker")
    st.warning("This feature is under development.")
    st.text_area("Your Notes:", height=300, placeholder="Start typing your notes here...")
    st.button("Save Note (Coming Soon!)", disabled=True)

elif selected_feature == "Visualizer (WIP)":
    st.header("📊 Visualizer")
    st.warning("This feature is under development.")
    st.info("Example: 'Plot a bar chart of sales data for Q1, Q2, Q3, Q4: 100, 150, 120, 180'")
    st.text_input("Describe your visualization:", placeholder="e.g., 'Show me a pie chart of my expenses: rent 500, food 200, transport 100'")
    st.button("Generate Visualization (Coming Soon!)", disabled=True)

elif selected_feature == "PDF Tools (WIP)":
    st.header("📄 PDF Tools")
    st.warning("This feature is under development.")
    st.subheader("1. PDF Summarizer")
    uploaded_pdf_summary = st.file_uploader("Upload a PDF to summarize:", type="pdf", key="pdf_summary_uploader")
    if uploaded_pdf_summary:
        st.info("PDF summarization will extract text and use Gemini.")
        if st.button("Summarize PDF (Coming Soon!)", disabled=True):
            st.write("Processing PDF...")

elif selected_feature == "About":
    st.header("ℹ️ About This App")
    st.info("This is a Streamlit application powered by Google's Gemini API, exposed to the internet via ngrok.")
    st.markdown("---")
    st.subheader("Key Technologies:")
    st.markdown("- **Streamlit:** For rapid web application development.")
    st.markdown("- **Google Gemini API:** For generative AI capabilities.")
    st.markdown("- **ngrok:** To create secure, public URLs for your local app.")
    st.markdown("- **python-dotenv:** For secure API key management.")
    st.markdown("- **gTTS:** For basic audio output.")
    st.markdown("---")
    st.subheader("Developed by:")
    st.markdown("Your Name/Organization (Optional)")
    st.markdown("Feel free to contribute or suggest improvements!")

2025-08-02 05:54:36.777 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-02 05:54:36.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-02 05:54:36.779 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-02 05:54:38.982 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-02 05:54:38.983 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-02 05:54:38.985 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-02 05:54:38.986 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-02 05:54:38.989 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Your Streamlit app is publicly available at: NgrokTunnel: "https://dbb4cbbc7369.ngrok-free.app" -> "http://localhost:8501"
